In [77]:
# import libraries
import os
import caer
import canaro
import numpy as np
import cv2 as cv
import gc
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
# Input picture and model initialization variables
IMG_SIZE = (80, 80)
channels = 1
char_path = r'../input/the-simpsons-characters-dataset/simpsons_dataset'
BATCH_SIZE = 32
EPOCHS = 10

In [79]:
# create dictionary of characters and image count
char_dict = {}

for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path, char)))
    
# Sort the dictionary in descending order
char_dict = caer.sort_dict(char_dict, descending = True)

# print(char_dict)

In [80]:
# obtain name of characters
characters = []
count = 0
for i in char_dict:
    characters.append(i[0])
    count += 1
    if count >= 10:
        break
        
# print(characters)

In [1]:
# create overall dataset
train = caer.preprocess_from_dir(char_path, characters, channels=channels, IMG_SIZE=IMG_SIZE, isShuffle=True)
;

NameError: name 'caer' is not defined

In [82]:
# obtain features and associated labels
featureSet, labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)

# normalize the feature set to  be between (0,1)
featureSet = caer.normalize(featureSet)

# one hot encode the label for efficiency
labels = to_categorical(labels, len(characters))

In [89]:
# split into training and validation
x_train, x_val, y_train, y_val = train_test_split(featureSet, labels)

In [90]:
# delete any unneccessary data (optinal)
#del train
#del featureSet
#del labels
#gc.collect()

In [94]:
# create image data generator pipeline
datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)

In [99]:
# Sepcifying our model
model = canaro.models.createSimpsonsModel(IMG_SIZE=IMG_SIZE, 
                                          channels=channels, 
                                          output_dim=len(characters), 
                                          loss='binary_crossentropy', 
                                          decay=1e-7, 
                                          learning_rate=0.001, 
                                          momentum=0.9,
                                          nesterov=True)

model.summary()
;

ValueError: Input tensors to a Functional must come from `tf.keras.Input`. Received: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7f741b85cc10>> (missing previous layer metadata).

In [ ]:
# Learning Rate Scheduler
callback_list = [LearningRateScheduler(canaro.lr_scheduler)]

In [ ]:
# train the model on training data
training = model.fit(train_gen,
                    steps_per_epoch = len(x_train)//BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data = (x.val, y_val),
                    validation_steps = len(y_val)//BATCH_SIZE,
                    callbacks = callbacks_list)

In [ ]:
# image preparation function/pipeline
def prepare(img:):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, IMG_SIZE)
    img = caer.reshape(img, IMG_SIZE, channels)
    return img

## Testing the model ##
test_path = r'../input/the-simpsons-characters-dataset/simpsons_dataset/homer_simpson/pic_0002.jpg'
img = cv.imread(test_path)
predictions = model.predict(prepare(img))
print(characters[np.argmax(predictions[0])])
